In [110]:
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation,TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
from collections import Counter
import numpy as np
from nltk.stem import PorterStemmer
#from gensim.models.doc2vec import Doc2Vec,TaggedDocument

In [79]:
df = pd.read_csv('restaurants.csv')
df = df.drop(columns=['Unnamed: 0'], errors='ignore')
df2 = df.sample(20)

In [80]:
# Tokenizer
def tokenizer(df):
    df['tokens'] = df['text'].apply(lambda text: word_tokenize(str(text)))
    return df

df2 = tokenizer(df2)
df2.head()

,review_id,user_id,business_id,stars_x,text,longitude,latitude,tokens
213499,xc2cZGN6Kx5ZOSjnn-5gig,ODLSDc5wF8K5fFssa5gVYQ,4hv1n1VBzMvE-T4UtIcOcw,5.0,Got a gift basket here for a friend's housewar...,-75.170065,39.951311,"[Got, a, gift, basket, here, for, a, friend, '..."
157810,5KqP95hdy7MvU1Vt_Premw,E1_MGWks4c-wEy8tJQCXHw,e3Y3hDpwHc9RmQlJtIgHuw,3.0,Up and down. Nice seat by the window with Ritt...,-75.173645,39.949913,"[Up, and, down, ., Nice, seat, by, the, window..."
66176,JCFcvztoMFBSeDCrGJJRYA,bEGLhmfZB3wgVZcn9Aswwg,dSPA4yql9vUwnYRxHiVtHA,3.0,You won't find the most amazing sushi or servi...,-75.163995,39.946685,"[You, wo, n't, find, the, most, amazing, sushi..."
275134,Z4S1Sc8bPfJxt4CQRbkcow,ssRQSJrQ_ApTByPwZGc2tg,8ohaL9a02bWZDxOxfZCTxw,4.0,I have been eating here for three years and th...,-75.167895,39.951265,"[I, have, been, eating, here, for, three, year..."
296664,hthfI_-3CNjmCYFWxlAwUw,AjA0sLYdZVj-NRMNYeNOnQ,xKobpA4w0P_J2jFMP7wcnw,5.0,Met a friend here on a particularly euphoric s...,-75.135053,39.977387,"[Met, a, friend, here, on, a, particularly, eu..."


In [81]:
# Number of sentences
def count_sentences(df):
    df['sentences'] = df['text'].apply(lambda text: sent_tokenize(str(text)))
    df['nr_sentences'] = df['sentences'].apply(lambda text: len(text))
    return df

df2 = count_sentences(df2)
df2.head()

,review_id,user_id,business_id,stars_x,text,longitude,latitude,tokens,sentences,nr_sentences
213499,xc2cZGN6Kx5ZOSjnn-5gig,ODLSDc5wF8K5fFssa5gVYQ,4hv1n1VBzMvE-T4UtIcOcw,5.0,Got a gift basket here for a friend's housewar...,-75.170065,39.951311,"[Got, a, gift, basket, here, for, a, friend, '...",[Got a gift basket here for a friend's housewa...,2
157810,5KqP95hdy7MvU1Vt_Premw,E1_MGWks4c-wEy8tJQCXHw,e3Y3hDpwHc9RmQlJtIgHuw,3.0,Up and down. Nice seat by the window with Ritt...,-75.173645,39.949913,"[Up, and, down, ., Nice, seat, by, the, window...","[Up and down., Nice seat by the window with Ri...",13
66176,JCFcvztoMFBSeDCrGJJRYA,bEGLhmfZB3wgVZcn9Aswwg,dSPA4yql9vUwnYRxHiVtHA,3.0,You won't find the most amazing sushi or servi...,-75.163995,39.946685,"[You, wo, n't, find, the, most, amazing, sushi...",[You won't find the most amazing sushi or serv...,20
275134,Z4S1Sc8bPfJxt4CQRbkcow,ssRQSJrQ_ApTByPwZGc2tg,8ohaL9a02bWZDxOxfZCTxw,4.0,I have been eating here for three years and th...,-75.167895,39.951265,"[I, have, been, eating, here, for, three, year...",[I have been eating here for three years and t...,6
296664,hthfI_-3CNjmCYFWxlAwUw,AjA0sLYdZVj-NRMNYeNOnQ,xKobpA4w0P_J2jFMP7wcnw,5.0,Met a friend here on a particularly euphoric s...,-75.135053,39.977387,"[Met, a, friend, here, on, a, particularly, eu...",[Met a friend here on a particularly euphoric ...,17


In [82]:
#porter stemmer

def stemmatization(df):
    porter = PorterStemmer()
    stemmatization = []

    for phrase in df['tokens']:
        stemmars = []
        for w in phrase:
            stemmars.append(porter.stem(w))
        
        stemmatization.append(stemmars)

    df['stemmatization'] = stemmatization
    return df

df2 = stemmatization(df2)
df2.head()

,review_id,user_id,business_id,stars_x,text,longitude,latitude,tokens,sentences,nr_sentences,stemmatization
213499,xc2cZGN6Kx5ZOSjnn-5gig,ODLSDc5wF8K5fFssa5gVYQ,4hv1n1VBzMvE-T4UtIcOcw,5.0,Got a gift basket here for a friend's housewar...,-75.170065,39.951311,"[Got, a, gift, basket, here, for, a, friend, '...",[Got a gift basket here for a friend's housewa...,2,"[got, a, gift, basket, here, for, a, friend, '..."
157810,5KqP95hdy7MvU1Vt_Premw,E1_MGWks4c-wEy8tJQCXHw,e3Y3hDpwHc9RmQlJtIgHuw,3.0,Up and down. Nice seat by the window with Ritt...,-75.173645,39.949913,"[Up, and, down, ., Nice, seat, by, the, window...","[Up and down., Nice seat by the window with Ri...",13,"[up, and, down, ., nice, seat, by, the, window..."
66176,JCFcvztoMFBSeDCrGJJRYA,bEGLhmfZB3wgVZcn9Aswwg,dSPA4yql9vUwnYRxHiVtHA,3.0,You won't find the most amazing sushi or servi...,-75.163995,39.946685,"[You, wo, n't, find, the, most, amazing, sushi...",[You won't find the most amazing sushi or serv...,20,"[you, wo, n't, find, the, most, amaz, sushi, o..."
275134,Z4S1Sc8bPfJxt4CQRbkcow,ssRQSJrQ_ApTByPwZGc2tg,8ohaL9a02bWZDxOxfZCTxw,4.0,I have been eating here for three years and th...,-75.167895,39.951265,"[I, have, been, eating, here, for, three, year...",[I have been eating here for three years and t...,6,"[i, have, been, eat, here, for, three, year, a..."
296664,hthfI_-3CNjmCYFWxlAwUw,AjA0sLYdZVj-NRMNYeNOnQ,xKobpA4w0P_J2jFMP7wcnw,5.0,Met a friend here on a particularly euphoric s...,-75.135053,39.977387,"[Met, a, friend, here, on, a, particularly, eu...",[Met a friend here on a particularly euphoric ...,17,"[met, a, friend, here, on, a, particularli, eu..."


In [83]:
def lemmatization(df):
    lemmatizer = WordNetLemmatizer()
    lemmatization = []

    for phrase in df['tokens']:
        lemmars = []
        for w in phrase:
            lemmars.append(lemmatizer.lemmatize(w))
        
        lemmatization.append(lemmars)

    df['lemmatization'] = lemmatization
    return df

df2 = lemmatization(df2)
df2.head()

,review_id,user_id,business_id,stars_x,text,longitude,latitude,tokens,sentences,nr_sentences,stemmatization,lemmatization
213499,xc2cZGN6Kx5ZOSjnn-5gig,ODLSDc5wF8K5fFssa5gVYQ,4hv1n1VBzMvE-T4UtIcOcw,5.0,Got a gift basket here for a friend's housewar...,-75.170065,39.951311,"[Got, a, gift, basket, here, for, a, friend, '...",[Got a gift basket here for a friend's housewa...,2,"[got, a, gift, basket, here, for, a, friend, '...","[Got, a, gift, basket, here, for, a, friend, '..."
157810,5KqP95hdy7MvU1Vt_Premw,E1_MGWks4c-wEy8tJQCXHw,e3Y3hDpwHc9RmQlJtIgHuw,3.0,Up and down. Nice seat by the window with Ritt...,-75.173645,39.949913,"[Up, and, down, ., Nice, seat, by, the, window...","[Up and down., Nice seat by the window with Ri...",13,"[up, and, down, ., nice, seat, by, the, window...","[Up, and, down, ., Nice, seat, by, the, window..."
66176,JCFcvztoMFBSeDCrGJJRYA,bEGLhmfZB3wgVZcn9Aswwg,dSPA4yql9vUwnYRxHiVtHA,3.0,You won't find the most amazing sushi or servi...,-75.163995,39.946685,"[You, wo, n't, find, the, most, amazing, sushi...",[You won't find the most amazing sushi or serv...,20,"[you, wo, n't, find, the, most, amaz, sushi, o...","[You, wo, n't, find, the, most, amazing, sushi..."
275134,Z4S1Sc8bPfJxt4CQRbkcow,ssRQSJrQ_ApTByPwZGc2tg,8ohaL9a02bWZDxOxfZCTxw,4.0,I have been eating here for three years and th...,-75.167895,39.951265,"[I, have, been, eating, here, for, three, year...",[I have been eating here for three years and t...,6,"[i, have, been, eat, here, for, three, year, a...","[I, have, been, eating, here, for, three, year..."
296664,hthfI_-3CNjmCYFWxlAwUw,AjA0sLYdZVj-NRMNYeNOnQ,xKobpA4w0P_J2jFMP7wcnw,5.0,Met a friend here on a particularly euphoric s...,-75.135053,39.977387,"[Met, a, friend, here, on, a, particularly, eu...",[Met a friend here on a particularly euphoric ...,17,"[met, a, friend, here, on, a, particularli, eu...","[Met, a, friend, here, on, a, particularly, eu..."


In [89]:
#delete stop words

def del_stopwords(df):

    stopwords_ = stopwords.words('english')
    new_phrase =[]
    for phrase in df['tokens']:
        tokens = []
        for w in phrase:
        
            if w not in stopwords_:
                tokens.append(w)
        
        new_phrase.append(tokens)

    df['del_stopwords'] = new_phrase
    return df

df2 = del_stopwords(df2)
df2.head()

,review_id,user_id,business_id,stars_x,text,longitude,latitude,tokens,sentences,nr_sentences,stemmatization,lemmatization,del_stopwords
213499,xc2cZGN6Kx5ZOSjnn-5gig,ODLSDc5wF8K5fFssa5gVYQ,4hv1n1VBzMvE-T4UtIcOcw,5.0,Got a gift basket here for a friend's housewar...,-75.170065,39.951311,"[Got, a, gift, basket, here, for, a, friend, '...",[Got a gift basket here for a friend's housewa...,2,"[got, a, gift, basket, here, for, a, friend, '...","[Got, a, gift, basket, here, for, a, friend, '...","[Got, gift, basket, friend, 's, housewarming, ..."
157810,5KqP95hdy7MvU1Vt_Premw,E1_MGWks4c-wEy8tJQCXHw,e3Y3hDpwHc9RmQlJtIgHuw,3.0,Up and down. Nice seat by the window with Ritt...,-75.173645,39.949913,"[Up, and, down, ., Nice, seat, by, the, window...","[Up and down., Nice seat by the window with Ri...",13,"[up, and, down, ., nice, seat, by, the, window...","[Up, and, down, ., Nice, seat, by, the, window...","[Up, ., Nice, seat, window, Rittenhouse, Park,..."
66176,JCFcvztoMFBSeDCrGJJRYA,bEGLhmfZB3wgVZcn9Aswwg,dSPA4yql9vUwnYRxHiVtHA,3.0,You won't find the most amazing sushi or servi...,-75.163995,39.946685,"[You, wo, n't, find, the, most, amazing, sushi...",[You won't find the most amazing sushi or serv...,20,"[you, wo, n't, find, the, most, amaz, sushi, o...","[You, wo, n't, find, the, most, amazing, sushi...","[You, wo, n't, find, amazing, sushi, service, ..."
275134,Z4S1Sc8bPfJxt4CQRbkcow,ssRQSJrQ_ApTByPwZGc2tg,8ohaL9a02bWZDxOxfZCTxw,4.0,I have been eating here for three years and th...,-75.167895,39.951265,"[I, have, been, eating, here, for, three, year...",[I have been eating here for three years and t...,6,"[i, have, been, eat, here, for, three, year, a...","[I, have, been, eating, here, for, three, year...","[I, eating, three, years, food, consistently, ..."
296664,hthfI_-3CNjmCYFWxlAwUw,AjA0sLYdZVj-NRMNYeNOnQ,xKobpA4w0P_J2jFMP7wcnw,5.0,Met a friend here on a particularly euphoric s...,-75.135053,39.977387,"[Met, a, friend, here, on, a, particularly, eu...",[Met a friend here on a particularly euphoric ...,17,"[met, a, friend, here, on, a, particularli, eu...","[Met, a, friend, here, on, a, particularly, eu...","[Met, friend, particularly, euphoric, summerli..."


In [120]:
# bag of words
def bag_of_words(df):
    vectorizer = CountVectorizer()
    bags = []

    for sentence in df['sentences']:
        X = vectorizer.fit_transform(sentence).toarray()
        bags.append(X)

    df['bag_of_words'] = bags
    return df

df2= bag_of_words(df2)
df2.head()

,review_id,user_id,business_id,stars_x,text,longitude,latitude,tokens,sentences,nr_sentences,stemmatization,lemmatization,del_stopwords,bag_of_words,n_grams,hot_vectors
213499,xc2cZGN6Kx5ZOSjnn-5gig,ODLSDc5wF8K5fFssa5gVYQ,4hv1n1VBzMvE-T4UtIcOcw,5.0,Got a gift basket here for a friend's housewar...,-75.170065,39.951311,"[Got, a, gift, basket, here, for, a, friend, '...",[Got a gift basket here for a friend's housewa...,2,"[got, a, gift, basket, here, for, a, friend, '...","[Got, a, gift, basket, here, for, a, friend, '...","[Got, gift, basket, friend, 's, housewarming, ...","[[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,...","[[0.28867513459481287, 0.0, 0.2886751345948128...","[[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,..."
157810,5KqP95hdy7MvU1Vt_Premw,E1_MGWks4c-wEy8tJQCXHw,e3Y3hDpwHc9RmQlJtIgHuw,3.0,Up and down. Nice seat by the window with Ritt...,-75.173645,39.949913,"[Up, and, down, ., Nice, seat, by, the, window...","[Up and down., Nice seat by the window with Ri...",13,"[up, and, down, ., nice, seat, by, the, window...","[Up, and, down, ., Nice, seat, by, the, window...","[Up, ., Nice, seat, window, Rittenhouse, Park,...","[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
66176,JCFcvztoMFBSeDCrGJJRYA,bEGLhmfZB3wgVZcn9Aswwg,dSPA4yql9vUwnYRxHiVtHA,3.0,You won't find the most amazing sushi or servi...,-75.163995,39.946685,"[You, wo, n't, find, the, most, amazing, sushi...",[You won't find the most amazing sushi or serv...,20,"[you, wo, n't, find, the, most, amaz, sushi, o...","[You, wo, n't, find, the, most, amazing, sushi...","[You, wo, n't, find, amazing, sushi, service, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2357022...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
275134,Z4S1Sc8bPfJxt4CQRbkcow,ssRQSJrQ_ApTByPwZGc2tg,8ohaL9a02bWZDxOxfZCTxw,4.0,I have been eating here for three years and th...,-75.167895,39.951265,"[I, have, been, eating, here, for, three, year...",[I have been eating here for three years and t...,6,"[i, have, been, eat, here, for, three, year, a...","[I, have, been, eating, here, for, three, year...","[I, eating, three, years, food, consistently, ...","[[0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0....","[[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,..."
296664,hthfI_-3CNjmCYFWxlAwUw,AjA0sLYdZVj-NRMNYeNOnQ,xKobpA4w0P_J2jFMP7wcnw,5.0,Met a friend here on a particularly euphoric s...,-75.135053,39.977387,"[Met, a, friend, here, on, a, particularly, eu...",[Met a friend here on a particularly euphoric ...,17,"[met, a, friend, here, on, a, particularli, eu...","[Met, a, friend, here, on, a, particularly, eu...","[Met, friend, particularly, euphoric, summerli...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [121]:
#hot vectors
def hot_vectors(df):
    vectorizer = CountVectorizer(binary=True)
    bags = []

    for sentence in df['sentences']:
        X = vectorizer.fit_transform(sentence).toarray()
        bags.append(X)

    df['hot_vectors'] = bags
    return df

df2= hot_vectors(df2)
df2.head()

,review_id,user_id,business_id,stars_x,text,longitude,latitude,tokens,sentences,nr_sentences,stemmatization,lemmatization,del_stopwords,bag_of_words,n_grams,hot_vectors
213499,xc2cZGN6Kx5ZOSjnn-5gig,ODLSDc5wF8K5fFssa5gVYQ,4hv1n1VBzMvE-T4UtIcOcw,5.0,Got a gift basket here for a friend's housewar...,-75.170065,39.951311,"[Got, a, gift, basket, here, for, a, friend, '...",[Got a gift basket here for a friend's housewa...,2,"[got, a, gift, basket, here, for, a, friend, '...","[Got, a, gift, basket, here, for, a, friend, '...","[Got, gift, basket, friend, 's, housewarming, ...","[[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,...","[[0.28867513459481287, 0.0, 0.2886751345948128...","[[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,..."
157810,5KqP95hdy7MvU1Vt_Premw,E1_MGWks4c-wEy8tJQCXHw,e3Y3hDpwHc9RmQlJtIgHuw,3.0,Up and down. Nice seat by the window with Ritt...,-75.173645,39.949913,"[Up, and, down, ., Nice, seat, by, the, window...","[Up and down., Nice seat by the window with Ri...",13,"[up, and, down, ., nice, seat, by, the, window...","[Up, and, down, ., Nice, seat, by, the, window...","[Up, ., Nice, seat, window, Rittenhouse, Park,...","[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
66176,JCFcvztoMFBSeDCrGJJRYA,bEGLhmfZB3wgVZcn9Aswwg,dSPA4yql9vUwnYRxHiVtHA,3.0,You won't find the most amazing sushi or servi...,-75.163995,39.946685,"[You, wo, n't, find, the, most, amazing, sushi...",[You won't find the most amazing sushi or serv...,20,"[you, wo, n't, find, the, most, amaz, sushi, o...","[You, wo, n't, find, the, most, amazing, sushi...","[You, wo, n't, find, amazing, sushi, service, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2357022...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,..."
275134,Z4S1Sc8bPfJxt4CQRbkcow,ssRQSJrQ_ApTByPwZGc2tg,8ohaL9a02bWZDxOxfZCTxw,4.0,I have been eating here for three years and th...,-75.167895,39.951265,"[I, have, been, eating, here, for, three, year...",[I have been eating here for three years and t...,6,"[i, have, been, eat, here, for, three, year, a...","[I, have, been, eating, here, for, three, year...","[I, eating, three, years, food, consistently, ...","[[0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0....","[[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,..."
296664,hthfI_-3CNjmCYFWxlAwUw,AjA0sLYdZVj-NRMNYeNOnQ,xKobpA4w0P_J2jFMP7wcnw,5.0,Met a friend here on a particularly euphoric s...,-75.135053,39.977387,"[Met, a, friend, here, on, a, particularly, eu...",[Met a friend here on a particularly euphoric ...,17,"[met, a, friend, here, on, a, particularli, eu...","[Met, a, friend, here, on, a, particularly, eu...","[Met, friend, particularly, euphoric, summerli...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [116]:
#n_grams
def n_grams(df):
    vectorizer = TfidfVectorizer(ngram_range=(2, 2))
    grams = []

    for sentence in df['sentences']:
        X = vectorizer.fit_transform(sentence).toarray()
        grams.append(X)

    df['n_grams'] = grams
    return df

df2= n_grams(df2)
df2.head()


,review_id,user_id,business_id,stars_x,text,longitude,latitude,tokens,sentences,nr_sentences,stemmatization,lemmatization,del_stopwords,bag_of_words,n_grams
213499,xc2cZGN6Kx5ZOSjnn-5gig,ODLSDc5wF8K5fFssa5gVYQ,4hv1n1VBzMvE-T4UtIcOcw,5.0,Got a gift basket here for a friend's housewar...,-75.170065,39.951311,"[Got, a, gift, basket, here, for, a, friend, '...",[Got a gift basket here for a friend's housewa...,2,"[got, a, gift, basket, here, for, a, friend, '...","[Got, a, gift, basket, here, for, a, friend, '...","[Got, gift, basket, friend, 's, housewarming, ...","[[1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,...","[[0.28867513459481287, 0.0, 0.2886751345948128..."
157810,5KqP95hdy7MvU1Vt_Premw,E1_MGWks4c-wEy8tJQCXHw,e3Y3hDpwHc9RmQlJtIgHuw,3.0,Up and down. Nice seat by the window with Ritt...,-75.173645,39.949913,"[Up, and, down, ., Nice, seat, by, the, window...","[Up and down., Nice seat by the window with Ri...",13,"[up, and, down, ., nice, seat, by, the, window...","[Up, and, down, ., Nice, seat, by, the, window...","[Up, ., Nice, seat, window, Rittenhouse, Park,...","[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
66176,JCFcvztoMFBSeDCrGJJRYA,bEGLhmfZB3wgVZcn9Aswwg,dSPA4yql9vUwnYRxHiVtHA,3.0,You won't find the most amazing sushi or servi...,-75.163995,39.946685,"[You, wo, n't, find, the, most, amazing, sushi...",[You won't find the most amazing sushi or serv...,20,"[you, wo, n't, find, the, most, amaz, sushi, o...","[You, wo, n't, find, the, most, amazing, sushi...","[You, wo, n't, find, amazing, sushi, service, ...","[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2357022..."
275134,Z4S1Sc8bPfJxt4CQRbkcow,ssRQSJrQ_ApTByPwZGc2tg,8ohaL9a02bWZDxOxfZCTxw,4.0,I have been eating here for three years and th...,-75.167895,39.951265,"[I, have, been, eating, here, for, three, year...",[I have been eating here for three years and t...,6,"[i, have, been, eat, here, for, three, year, a...","[I, have, been, eating, here, for, three, year...","[I, eating, three, years, food, consistently, ...","[[0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0, 0...."
296664,hthfI_-3CNjmCYFWxlAwUw,AjA0sLYdZVj-NRMNYeNOnQ,xKobpA4w0P_J2jFMP7wcnw,5.0,Met a friend here on a particularly euphoric s...,-75.135053,39.977387,"[Met, a, friend, here, on, a, particularly, eu...",[Met a friend here on a particularly euphoric ...,17,"[met, a, friend, here, on, a, particularli, eu...","[Met, a, friend, here, on, a, particularly, eu...","[Met, friend, particularly, euphoric, summerli...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
